#**TODO**

Refine keywords : strip \n, ", ' etc ...

Handle ill-encoded keywords : utf-8 ...?

#**Upload Files**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving journal_of_development_economics_final.xlsx to journal_of_development_economics_final (1).xlsx
Saving journal_of_international_economics_final.xlsx to journal_of_international_economics_final (1).xlsx
Saving journal_of_world_trade_final.xlsx to journal_of_world_trade_final (1).xlsx
Saving the_world_economy_final.xlsx to the_world_economy_final (1).xlsx
Saving world_development_final.xlsx to world_development_final (1).xlsx


#**Import Modules**

In [0]:
import openpyxl

In [0]:
!pip install xlsxwriter
import xlsxwriter

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
import gc

#**Read and Process Data**

In [0]:
def pair_to_vector(idx, length):
    res = [0] * length
    for i in idx:
        res[i] = 1
    return res

In [0]:
FILENAMES = ["journal_of_international_economics_final.xlsx", "the_world_economy_final.xlsx", "journal_of_world_trade_final.xlsx", "journal_of_development_economics_final.xlsx", "world_development_final.xlsx"]

journals = []
keywords = []
papers = []
keyword_paper_pair = []
idx_journals = [0]
idx_yrs = [["80s"], ["90s"], ["00s"], ["10s"]]
curr_idx = -1

for i in range(len(FILENAMES)):
    # open workbook
    wb = openpyxl.load_workbook(FILENAMES[i])
    ws = wb.active
    MAXROW = ws.max_row
    MAXCOL = ws.max_column
    curr = 0

    journals.append(ws.cell(row=2, column=1).value)

    for r in ws.rows:
        # ignore header
        if(r[0].row == 1):
            continue
        # halt loop
        if(r[0].row > MAXROW):
            break
        # update cursor and fetch year index
        if(isinstance(r[2].value, str)):
            curr = r[2].value
            papers.append(curr)
            curr_idx += 1

            voloryear = int(r[1].value)
            journal_vol_2_year = [[10, 28, 50, 80], [1980, 1990, 2000, 2010], [14, 24, 34, 44], [7, 32, 61, 91], [8, 18, 28, 38]]
            for j in range(len(journal_vol_2_year[i])):
                if(j == 3 and voloryear >= journal_vol_2_year[i][j]):
                    idx_yrs[j].append(curr_idx) 
                elif(voloryear >= journal_vol_2_year[i][j] and voloryear < journal_vol_2_year[i][j + 1]):
                    idx_yrs[j].append(curr_idx)
                    break

        # row corresponds to a keyword
        if(isinstance(r[3].value, str)):
            # duplicate exists
            if(r[3].value.lower() in keywords):
                keyword_paper_pair[keywords.index(r[3].value.lower())].append(curr_idx)
            # duplicate doesn't exist
            else:
                keywords.append(r[3].value.lower())
                keyword_paper_pair.append([curr_idx])
    
    # fetch journal index
    idx_journals.append(curr_idx + 1)
        
# encode keyword-paper pair to vector
keyword_vector = []
for i in range(len(keyword_paper_pair)):
    keyword_vector.append(pair_to_vector(keyword_paper_pair[i], curr_idx + 1))

wb.close()

In [0]:
len(papers)

5664

In [0]:
len(keywords)

9942

In [0]:
len(keyword_vector)

9942

#**Write Paper-Keyword Matrix to Excel**

In [0]:
wb = xlsxwriter.Workbook("paper-keyword_mat.xlsx")
ws = wb.add_worksheet()

for i in range(len(papers)):
    ws.write(i + 1, 0, papers[i])
for j in range(len(keyword_vector)):
    ws.write(0, j + 1, keywords[j])
    for i in range(len(keyword_vector[j])):
        ws.write(i + 1, j + 1, keyword_vector[j][i])

wb.close()

#**Write Paper-Keyword Matrix to TXT**

In [0]:
f = open("paper-keyword_mat.txt", "w")
f.write("DL NR=" + str(len(papers)) + " NC=" + str(len(keywords)) + "\n")
f.write("Format=Fullmatrix\n")
f.write("Row labels:\n")
f.writelines(" ".join(map(str, list(range(len(papers))))) + "\n")
f.write("Col labels:\n")
keywords_short = []
for i in range(len(keywords)):
    kwd = keywords[i].strip()
    if(len(kwd) > 18):
        kwd = kwd[:18]
    kwd = "\"" + kwd + "\""
    keywords_short.append(kwd)
f.writelines(" ".join(keywords_short) + "\n")
f.write("Data:\n")
keyword_vector_t = list(map(list, zip(*keyword_vector)))
for line in keyword_vector_t:
    f.writelines(" ".join(map(str, line)) + "\n")

#**Paper-Keyword Matrix to 1-Mode Matrix**

In [0]:
keyword_1-mode_mat = np.dot(np.array(keyword_vector), np.transpose(np.array(keyword_vector)))

#**Sort Keywords by Frequency**

**Make a Dataframe**

In [0]:
df = pd.DataFrame(keyword_vector)
df = df.transpose()
df.index = papers
df.columns = keywords

In [0]:
df

,agricultural trade,gravity,trade costs,discrete choice,productivity,comparative advantage,demand complementarities,goods & services,oligopoly,firm-level exports,quality,banking crises,china,financial frictions,lender of last resort,agglomeration,assignment models,cities,inflation,impact of globalization,strategic interactions,market structure,phillips curve,resource insecurity,interstate disputes,conflict,arming,trade openness,sovereign default,corporate borrowing,interest rate spreads,fiscal policy,multinational production,non-neutral technologies,labor shares,non-homothetic preferences,skill premium,per capita income,structural change,international trade,...,ngo program,education management,nonformal,low-cost,technological capability,technological learning,technological complexity,farm equipment,carbon dioxide emission,greenhouse gas,r & d,researchers,development strategy,land resettlement,resource economics,quasi-fiscal deficits,capital adequacy,greenhouse effect,cross-national,education finance,small growers,trypanosomiasis,tsetse flies,fuelwood shortage,household fuel use,fuelwood collection,share-cropping,interlinkage model,financial reform,measurement indices,forest size,historical trends,united nations conferences,influence,unced,world food conference,stakeholder analysis,research policy,rural mechanization,income-distribution
"Technology, ecology and agricultural trade",1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
One way to the top: How services boost the demand for goods,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A quantitative model of international lending of last resort,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
The comparative advantage of cities,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Globalization, market structure and inflation dynamics",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"The paths to rain forest destruction: Crossnational patterns of tropical deforestation, 1975–1990",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
Settler households and land-use patterns in the Amazon frontier: Farm-level evidence from Ecuador,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"The frontiers of influence: NGO lobbying at the 1974 World Food Conference, the 1992 Earth Summit and beyond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
"Stakeholders, science and decision making for poverty-focused rural mechanization research and development",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0


**Write Ranks to Excel**

In [0]:
def initial(s):
    res = ''
    for a in s.split(' '):
        res += a[0]
    return res

In [0]:
wb = openpyxl.Workbook()
wb.save("kwd_rank.xlsx")

# total
freq_tot = df.sum(axis=0).tolist()
freq_tot_mat = []
for i in range(len(freq_tot)):
    freq_tot_mat.append([keywords[i], freq_tot[i]])
freq_tot_mat.sort(key=lambda x: x[1])
freq_tot_mat.reverse()
#freq_tot_mat = freq_tot_mat[:30]
wb.create_sheet("total")
ws = wb.get_sheet_by_name("total")
for j in range(len(freq_tot_mat)):
        ws.cell(row=j + 1, column = 1).value = freq_tot_mat[j][0]
        ws.cell(row=j + 1, column = 2).value = freq_tot_mat[j][1]
wb.save("kwd_rank.xlsx")

# by year
for i in range(len(idx_yrs)):
    yr = idx_yrs[i][0]
    df_yr = df.iloc[idx_yrs[i][1:]]
    freq_yr = df_yr.sum(axis=0).tolist()
    freq_yr_mat = []
    for j in range(len(freq_yr)):
        freq_yr_mat.append([keywords[j], freq_yr[j]])
    freq_yr_mat.sort(key=lambda x: x[1])
    freq_yr_mat.reverse()
    freq_yr_mat = freq_yr_mat[:30]
    wb.create_sheet(yr)
    ws = wb.get_sheet_by_name(yr)
    for j in range(len(freq_yr_mat)):
        ws.cell(row=j + 1, column = 1).value = freq_yr_mat[j][0]
        ws.cell(row=j + 1, column = 2).value = freq_yr_mat[j][1]
    wb.save("kwd_rank.xlsx")

# by journal
for i in range(len(idx_journals) - 1):
    df_journal = df[idx_journals[i]:idx_journals[i + 1]]
    freq_journal = df_journal.sum(axis=0).tolist()
    freq_journal_mat = []
    for j in range(len(freq_journal)):
        freq_journal_mat.append([keywords[j], freq_journal[j]])
    freq_journal_mat.sort(key=lambda x: x[1])
    freq_journal_mat.reverse()
    freq_journal_mat = freq_journal_mat[:30]
    wb.create_sheet(initial(journals[i]))
    ws = wb.get_sheet_by_name(initial(journals[i]))
    for j in range(len(freq_journal_mat)):
        ws.cell(row=j + 1, column = 1).value = freq_journal_mat[j][0]
        ws.cell(row=j + 1, column = 2).value = freq_journal_mat[j][1]
    wb.save("kwd_rank.xlsx")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: Depre

KeyboardInterrupt: ignored

**For better Readability**

In [0]:
wb = openpyxl.Workbook()
wb.save("kwd_rank_readable.xlsx")

freq_tot = df.sum(axis=0).tolist()
freq_tot_mat = []
for i in range(len(freq_tot)):
    freq_tot_mat.append([keywords[i], freq_tot[i]])
freq_tot_mat.sort(key=lambda x: x[1])
freq_tot_mat.reverse()
sheetno = 1
wb.create_sheet(str(sheetno))
ws = wb.get_sheet_by_name(str(sheetno))
rowno = 1
colno = 1
for j in range(len(freq_tot_mat)):
    if((j + 1) % 100 == 0):
        rowno = 1
        colno += 2
    if((j + 1) % 500 == 0):
        sheetno += 1
        rowno = 1
        colno = 1
        wb.create_sheet(str(sheetno))
        ws = wb.get_sheet_by_name(str(sheetno))
    ws.cell(row=rowno, column = colno).value = freq_tot_mat[j][0]
    ws.cell(row=rowno, column = colno + 1).value = freq_tot_mat[j][1]
    rowno += 1
wb.save("kwd_rank_readable.xlsx")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24:

#**Write Refined Paper-Keyword Matrix to Excel**

delete keywords that have less than 10 occurrences

In [0]:
df_over10 = df
df_over10.loc["SUM"] = df_over10.sum(axis=0).tolist()
df_over10 = df_over10.sort_values(by=["SUM"], axis=1, ascending=False)

idx_cut = 0
for i in range(len(df_over10.loc["SUM"])):
    if(df_over10.loc["SUM"][i] < 10):
        idx_cut = i
        break
df_over10 = df_over10[:-1]
df_over10 = df_over10.transpose()
df_over10 = df_over10[:idx_cut].transpose()

df_over10.to_excel("paper-keyword_mat_over10.xlsx")

#**Download Files**

In [0]:
files.download("kwd_rank.xlsx")

In [0]:
files.download("kwd_rank_readable.xlsx")

In [0]:
files.download("paper-keyword_mat.txt")

In [0]:
files.download("paper-keyword_mat.xlsx")

In [0]:
files.download("paper-keyword_mat_over10.xlsx")